# Annual Maxima（CSV 處理流程）


本 Notebook 直接以 pandas 操作原始 CSV，計算每個測站在不同累積時數下的年度最大雨量，並輸出成寬表。


In [2]:
# 基本套件與路徑設定
from pathlib import Path

import numpy as np
import pandas as pd

DATA_DIR = Path("data")
CSV_PATH = DATA_DIR / "Rain_1998-2017.csv"
START_TS = "1998-01-01 00:00:00"
END_TS = "2017-12-31 23:00:00"
DURATIONS = [1, 2, 3, 6, 12, 24, 48, 72]

print(f"資料來源: {CSV_PATH}")
print(f"處理期間: {START_TS} ~ {END_TS}")


資料來源: data/Rain_1998-2017.csv
處理期間: 1998-01-01 00:00:00 ~ 2017-12-31 23:00:00


In [3]:
# 讀取雨量 CSV 並處理時間戳與特殊值
raw_df = pd.read_csv(
    CSV_PATH,
    dtype={"stno": "string", "yyyymmddhh": "string"},
)
df = raw_df.rename(columns={"stno": "station_no", "PP01": "rainfall_raw"}).copy()

rainfall_numeric = pd.to_numeric(df["rainfall_raw"], errors="coerce")
df["rainfall_value"] = rainfall_numeric.where(rainfall_numeric != -9996, np.nan)
df["rainfall_clean"] = df["rainfall_value"].clip(lower=0).fillna(0)

stamp = df["yyyymmddhh"].fillna("")
dates = pd.to_datetime(stamp.str[:8], format="%Y%m%d", errors="coerce")
hours = pd.to_numeric(stamp.str[-2:], errors="coerce").fillna(0).astype(int)
rollover = hours == 24
hours = hours.where(~rollover, 0)
obs_ts = dates + pd.to_timedelta(hours, unit="h")
obs_ts.loc[rollover] = obs_ts.loc[rollover] + pd.Timedelta(days=1)
df["obs_ts"] = obs_ts

df = df.dropna(subset=["station_no", "obs_ts"]).reset_index(drop=True)
df.head()


,station_no,yyyymmddhh,rainfall_raw,rainfall_value,rainfall_clean,obs_ts
0,466880,2002010101,0.0,0.0,0.0,2002-01-01 01:00:00
1,466880,2002010102,0.0,0.0,0.0,2002-01-01 02:00:00
2,466880,2002010103,0.0,0.0,0.0,2002-01-01 03:00:00
3,466880,2002010104,0.0,0.0,0.0,2002-01-01 04:00:00
4,466880,2002010105,0.0,0.0,0.0,2002-01-01 05:00:00


In [5]:
# 建立完整 hourly grid，缺少的時間會填入 NaN 以利覆蓋率計算
stations = sorted(df["station_no"].unique())
full_range = pd.date_range(START_TS, END_TS, freq="H")
full_index = pd.MultiIndex.from_product(
    [stations, full_range], names=["station_no", "obs_ts"]
)

hourly = (
    df[["station_no", "obs_ts", "rainfall_value", "rainfall_clean"]]
    .groupby(["station_no", "obs_ts"], as_index=False)
    .agg({"rainfall_value": "last", "rainfall_clean": "last"})
    .set_index(["station_no", "obs_ts"])
    .reindex(full_index)
)

hourly["rainfall_clean"] = hourly["rainfall_clean"].fillna(0)
hourly["recorded_flag"] = (
    hourly["rainfall_value"].eq(-9998) | (hourly["rainfall_value"] >= 0)
).fillna(False)

EXPECTED_HOURS = len(full_range)
coverage = (
    hourly["recorded_flag"]
    .groupby(level="station_no")
    .agg(total_rows="size", recorded_rows="sum")
    .reset_index()
)
coverage["expected_records"] = EXPECTED_HOURS
coverage["pct_of_expected"] = coverage["recorded_rows"] * 100.0 / EXPECTED_HOURS
coverage["meets_95pct"] = coverage["pct_of_expected"] >= 95.0
coverage.sort_values("station_no").head()


/tmp/ipykernel_3514098/1477222278.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(START_TS, END_TS, freq="H")


,station_no,total_rows,recorded_rows,expected_records,pct_of_expected,meets_95pct
0,466880,175320,140255,175320,79.999430,False
1,466900,175320,175142,175320,99.898471,True
2,466910,175320,175319,175320,99.999430,True
3,466920,175320,175319,175320,99.999430,True
4,466930,175320,175319,175320,99.999430,True


In [7]:
# 依覆蓋率挑選達標測站並列出未達標清單
qualified = coverage.loc[coverage["meets_95pct"], "station_no"].tolist()
unqualified = coverage.loc[~coverage["meets_95pct"], ["station_no", "pct_of_expected"]]
print(f"達標測站：{len(qualified)} / {len(coverage)}")
print(f"範例達標測站：{qualified[:5]}")
print("未達標測站預覽：")
unqualified.sort_values("pct_of_expected").head()


達標測站：187 / 772
範例達標測站：['466900', '466910', '466920', '466930', '466940']
未達標測站預覽：


,station_no,pct_of_expected
771,c1I080,0.0
476,C1A740,0.0
470,C0Z330,0.0
469,C0Z320,0.0
468,C0Z310,0.0


In [9]:
# 以 pandas rolling 計算年度最大累積雨量
if not qualified:
    raise ValueError("沒有達標測站可供計算，請檢查覆蓋率條件。")

good_hourly = hourly.loc[qualified]
annual_frames = []

for hrs in DURATIONS:
    rolling = (
        good_hourly["rainfall_clean"]
        .groupby(level=0)
        .rolling(window=hrs, min_periods=hrs)
        .sum()
        .rename("window_sum")
    )
    rolling = rolling.droplevel(0)  # 移除 groupby 產生的重複 station 層
    rolling = rolling.reset_index()
    rolling = rolling.rename(columns={"level_0": "station_no", "level_1": "obs_ts"})
    rolling["year"] = rolling["obs_ts"].dt.year
    rolling = rolling.dropna(subset=["window_sum"])

    idx = rolling.groupby(["station_no", "year"])['window_sum'].idxmax()
    annual = rolling.loc[idx, ["station_no", "year", "obs_ts", "window_sum"]].copy()
    annual = annual.rename(columns={"obs_ts": "window_end_ts", "window_sum": "annual_max_mm"})
    annual["duration_hours"] = hrs
    annual_frames.append(annual)

annual_max = pd.concat(annual_frames, ignore_index=True)
annual_max["year"] = pd.to_datetime(annual_max["year"].astype(int).astype(str), format="%Y")
annual_max.sort_values(["station_no", "year", "duration_hours"]).head()


,station_no,year,window_end_ts,annual_max_mm,duration_hours
0,466900,1998-01-01,1998-06-04 19:00:00,46.5,1
3740,466900,1998-01-01,1998-06-04 19:00:00,70.5,2
7480,466900,1998-01-01,1998-06-04 20:00:00,81.0,3
11220,466900,1998-01-01,1998-10-15 23:00:00,145.0,6
14960,466900,1998-01-01,1998-10-16 02:00:00,207.0,12


In [10]:
annual_max.head()

,station_no,year,window_end_ts,annual_max_mm,duration_hours
0,466900,1998-01-01,1998-06-04 19:00:00,46.5,1
1,466900,1999-01-01,1999-06-17 18:00:00,36.5,1
2,466900,2000-01-01,2000-11-01 02:00:00,48.0,1
3,466900,2001-01-01,2001-09-05 21:00:00,59.0,1
4,466900,2002-01-01,2002-07-10 11:00:00,55.5,1
